In [1]:
import control
import foton
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal
from timeit import default_timer as timer
import yaml

import x1sim


# Load plant models
# plant_par1 = np.load("model_params/plant1.npz")
# plant_par2 = np.load("model_params/plant2.npz")
# ^Unstable
plant_par1 = np.load("model_params/plant1_no_m1.npz")
plant_par2 = np.load("model_params/plant2_no_m1.npz")
plant_par3 = np.load("model_params/plant3.npz")

# plant_par = np.load("model_params/test_plant.npz")

# Load controller (test only)
# controller_par = np.load("model_params/test_controller.npz")

# Load disturbance and noise
seis_par = np.load("model_params/seis_noise.npz")
sens_par = np.load("model_params/sens_noise.npz")

In [2]:
# Make simulation
arm = x1sim.x1sim()

sens       = arm.LIGOFilter('SENS')
ctrl       = arm.LIGOFilter('CTRL')
act        = arm.LIGOFilter('ACT')
plant      = arm.LIGOFilter('PLANT')
sens_noise = arm.LIGOFilter('SENS_NOISE')
seis_noise = arm.LIGOFilter('SEIS_NOISE')

sens.ramp_gain(1, 0)
act.ramp_gain(1, 0)

# Load plant models
arm.fm_set_zpk('PLANT', 'FM3', plant_par1['zs'], plant_par1['ps'], plant_par1['k'])
arm.fm_set_zpk('PLANT', 'FM4', plant_par2['zs'], plant_par2['ps'], float(plant_par2['k']))
arm.fm_set_zpk('PLANT', 'FM5', plant_par3['zs'], plant_par3['ps'], plant_par3['k'])

plant.turn_on('FM3')
plant.turn_on("FM4")
plant.turn_on("FM5")

plant.ramp_gain(1, 0)

# Load controllers
filter_file = foton.FilterFile("foton_files/L1LSC.txt")
srcl_module = filter_file["LSC_SRCL"]
fms_that_are_on = [1, 2, 3, 4, 7, 9, 10]

for fmnum in fms_that_are_on:
    z, p, k = srcl_module[fmnum - 1].get_zpk()
    arm.fm_set_zpk('CTRL', f'FM{fmnum}',  z, p, k)
    
for fmnum in fms_that_are_on:
    ctrl.turn_on(f"FM{fmnum}")

ctrl.ramp_gain(-0.4*10, 0)  # *10 from SRCL FM4.

# For test
# arm.fm_set_zpk('CTRL', 'FM3', controller_par['zs'], controller_par['ps'], controller_par['k'])
# ctrl.turn_on("FM3")
# ctrl.ramp_gain(-1, 0)

# Find unit PSD amplitude
fs = arm.sample_rate
unit_amplitude = np.sqrt(fs/2)

# Set seismic noises
arm.fm_set_zpk("SEIS_NOISE", "FM3", seis_par["zs"], seis_par["ps"], seis_par["k"])
seis_noise.turn_on("FM3")
seis_noise.ramp_gain(unit_amplitude, 0)

# Set sensor noise
# noise_par = np.load("model_params/noise.npz")
# arm.fm_set_zpk("SENS_NOISE", "FM3", sens_par["zs"], sens_par["ps"], sens_par["k"])
# sens_noise.turn_on("FM3")
sens_noise.ramp_gain(unit_amplitude*sens_par["k"], 0)  # Sensor noise is flat.
# sens_noise.ramp_gain(1, 0)

In [3]:
def dump_yaml(mdl, f, prefix=''):
    cfg = dict()

    var_names  = set(mdl.vars)
    fm_names   = set(mdl.filter_modules)
    var_names -= set(f'{x}_IN1' for x in fm_names)
    var_names -= set(f'{x}_IN2' for x in fm_names)
    var_names -= set(f'{x}_EXC' for x in fm_names)
    var_names -= set(f'{x}_OUT' for x in fm_names)
    var_names -= set(f'{x}_OUT16' for x in fm_names)
    var_names -= set(f'{x}_OUTPUT' for x in fm_names)
    var_names -= set(f'{x}_SWS' for x in fm_names)
    var_names -= set(f'{x}_SWSTAT' for x in fm_names)
    var_names  = set(x for x in var_names if not x.startswith(('ADC_WAIT', 'AWG_STAT')))
    var_names  = set(x for x in var_names if not x.startswith(('BURT_RESTORE', 'DAC_DUO_SET')))
    var_names  = set(x for x in var_names if not x.startswith(('DISABLE_REMOTE_IPC',)))
    var_names  = set(x for x in var_names if not x.startswith(('BIO_MON', 'CPU_METER')))
    var_names  = set(x for x in var_names if not x.startswith(('GDS_MON', 'IPC_TOTAL')))
    var_names  = set(x for x in var_names if not x.startswith(('OVERFLOW_ADC', 'STAT_ADC')))
    var_names -= set(('CPU_LONG_CYCLE', 'CYCLE'))
    var_names -= set(('DAC_ENABLE', 'DAQ_BYTE_CNT', 'DCU_ID', 'DIAG_RESET'))
    var_names -= set(('DIAG_WORD', 'EPICS_SYNC', 'FB_NET_STAT'))
    var_names -= set(('FE_STATUS', 'IPC_DIAG_RESET', 'IPC_STAT', 'OVERFLOW_RESET'))
    var_names -= set(('OV_ACCUM', 'PAD1', 'STARTGPSTIME', 'STATE_WORD'))
    var_names -= set(('TIME_DIAG', 'TIME_ERR', 'TP_CNT', 'USER_TIME', 'VME_RESET'))
    cfg['vars']= dict((prefix + k, mdl.read(k)) for k in var_names)

    cfg['filters'] = dict()
    for fm_name in fm_names:
        cfg['filters'][prefix + fm_name] = dict()
        cfg['filters'][prefix + fm_name]['switches'] = mdl.fm_get_switches(fm_name)
        for section_name in [f'FM{n+1}' for n in range(10)]:
            sos = mdl.fm_get_sos(fm_name, section_name, format='o').tolist()
            cfg['filters'][prefix + fm_name][section_name] = dict()
            cfg['filters'][prefix + fm_name][section_name]['coef'] = sos
            settings = mdl.fm_get_filter_settings(fm_name, section_name)
            cfg['filters'][prefix + fm_name][section_name]['settings'] = settings
    
    yaml.safe_dump(cfg, f, default_flow_style=None)

dof = "SRCL"

with open(f'x1sim_{dof}.yaml', 'w') as f:
    dump_yaml(arm, f)